In [ ]:
import pandas as pd
import numpy as np

def read_file():
    filename = input("Enter the file name (i.e. class1 for class1.txt): ") # Gọi tên file không cần đuôi.txt
    try:
        with open(filename+'.txt', 'r') as file:  # Mở filename phải có đuôi txt ở chế độ đọc
            print("Successfully opened", filename)  # In ra màn hình chuỗi "Successfully opened filename"
            lines = file.readlines() # Đọc và tách tất cả các dòng có trong file thành từng list
            data = [line.strip().split(",") for line in lines] # Tạo biến data xét từng dòng strip(xóa khoảng trắng), phân tách các phần tử trong 1 list thành nhiều list
            return pd.DataFrame(data),filename # In dữ liệu biến data + tên file
    except FileNotFoundError: # Giả định không tìm thấy file
        print("File not found.") # In ra màn hình chuỗi "File not found."
        return None # Không trả về giá trị nào


def analyze(data):
    valid_line = 0  # Số dòng hợp lệ
    invalid_line = 0 # Số dòng không hợp lệ
    student_answers = {} # Từ điển lưu trữ câu trả lời của sinh viên hợp lệ, với mã sinh viên là khóa và danh sách câu trả lời là giá trị
    not26value = {} # Từ điển lưu trữ các dòng có số lượng giá trị không phải 26
    invalid_N = {} # Từ điển lưu trữ các dòng có mã sinh viên không hợp lệ

    # Vòng lặp qua từng dòng trong DataFrame 'data'
    for index, row in data.iterrows():
        row = row.dropna() # Loại bỏ bất kỳ giá trị thiếu nào (NaN) trong dòng
        # Kiểm tra điều kiện để phân loại các dòng vào các loại hợp lệ và không hợp lệ
        if len(row) == 26 and row[0][0] == 'N' and row[0][1:9].isdigit(): # Nếu tổng dòng = 26 và ở ô đầu của cột 1 = 'N' và cột 1 là số thì
            valid_line += 1
            student_id = row[0]  # Lấy mã sinh viên
            answers = row.iloc[1:].tolist()  # Lấy câu trả lời dưới dạng danh sách
            student_answers[student_id] = answers  # Lưu mã sinh viên và câu trả lời vào dict
        elif len(row) != 26:
            not26value[row[0]] = row.iloc[1:26].tolist()
            invalid_line += 1
        elif row[0][0] != 'N' or not row[0][1:9].isdigit():
            invalid_N[row[0]] = row.iloc[1:26].tolist()
            invalid_line += 1

    # Trả về dữ liệu và số dòng hợp lệ và không hợp lệ dưới dạng tuple
    return student_answers, not26value, invalid_N, valid_line, invalid_line

#Hàm chấm điểm
def grades(student_answers):
    # Khai báo một bộ đáp án đúng mẫu
    answer_key = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D"
    answer_key = answer_key.split(',') # Chuyển đổi bộ đáp án đúng mẫu thành danh sách
    # Tạo các từ điển để lưu trữ kết quả chấm điểm của từng sinh viên
    scores = {}
    skip_question = {}
    wrong_answers = {}

     # Vòng lặp qua từng câu hỏi và câu trả lời tương ứng của sinh viên
    for student_id, answers in student_answers.items():
        score = 0
        for i, answer in enumerate(answers):

            if i not in skip_question:
                skip_question[i] = 0
                wrong_answers[i] = 0
            # Kiểm tra nếu chưa có thông tin về câu hỏi thứ i trong từ điển, thì khởi tạo giá trị ban đầu
            if answer == answer_key[i]: # Nếu câu trả lời đúng, cộng điểm
                score += 4
            elif answer == '':
                score += 0

                skip_question[i] += 1   # do i bắt đầu từ 0 đến len(answers)


            elif answer != answer_key[i] and answer != '': # Nếu câu trả lời sai và không bị bỏ qua, trừ điểm và cập nhật số câu trả lời sai
                score -= 1
                wrong_answers[i] +=1


        scores[student_id] = score   # Lưu điểm số của sinh viên vào từ điển
    # Trả về các từ điển chứa kết quả chấm điểm của từng sinh viên, số lần bỏ qua câu hỏi và số lượng câu trả lời sai
    return scores,skip_question,wrong_answers


# Gọi hàm để đọc file
data,filename = read_file()

# Đoạn code này thực hiện việc phân tích và xử lý dữ liệu đầu vào
if data is not None:
    # Phân tích dữ liệu từ DataFrame data
    student_answers, not26value, invalid_N, valid_line, invalid_line = analyze(data)
    # Tính điểm cho từng sinh viên từ các câu trả lời
    scores,skip_question, wrong_answers = grades(student_answers)
    print()
    print("**** ANALYZING ****")
    print()

    # Kiểm tra và in thông tin về các dòng dữ liệu không hợp lệ
    if not26value != {} or invalid_N != {}:
        if not26value != {}:
            print("Invalid lines of data: does not contain exactly 26 values:")
            for student_id, answers in not26value.items():
                print(student_id + ',', ','.join(answers))
        else: pass


        if invalid_N != {}:
            print("Invalid lines of data: N# is invalid:")
            for student_id, answers in invalid_N.items():
                print(student_id + ',', ','.join(answers))
        else: pass
    else:

        print("No errors found!")
    print()
    # In báo cáo tổng quan về dữ liệu và kết quả chấm điểm
    print("**** REPORT ****")

    print("Total valid lines of data:", valid_line)
    print("Total invalid lines of data:", invalid_line)

    count_high_scores = sum(score> 80 for score in scores.values())
    print("Total students with high scores (>80):", count_high_scores)

    average_score = round(np.mean(list(scores.values())), 3)
    print("Mean (average) score:", average_score)

    max_score = np.max(list(scores.values()))
    print("Max score:", max_score)

    min_score = np.min(list(scores.values()))
    print("Min score:", min_score)

    range_score = max_score-min_score
    print("Range of scores:",range_score)


    median_score = np.median(list(scores.values()))
    print("Median score:",median_score)

    # Tìm câu hỏi mà sinh viên trả lời nhiều nhất và sai nhiều nhất
    max_skip = np.max(list(skip_question.values()))
    most_skipped_questions = []
    for question, count in skip_question.items():
        if skip_question[question] == max_skip:
            skip_rate = round(count / len(student_answers), 3)
            most_skipped_questions.append(f"{question+1} - {count} - {skip_rate}")

    print("Question that most people skip:", ', '.join(most_skipped_questions))

    max_wrong = np.max(list(wrong_answers.values()))
    most_wrong_answers = []
    for answer, count in wrong_answers.items():
        if wrong_answers[answer] == max_wrong:
            wrong_rate = round(count / len(student_answers), 3)
            most_wrong_answers.append(f"{answer+1} - {count} - {wrong_rate}")

    print("Question that most people answer incorrectly:", ', '.join(most_wrong_answers))



    # Lưu kết quả chấm điểm vào file
    output_filename = filename +"_grade.txt"
    with open (output_filename,"w") as output_file:
        for student_id, score in scores.items():
            output_file.write(f"{student_id},{score}\n")


Enter the file name (i.e. class1 for class1.txt): class5
Successfully opened class5

**** ANALYZING ****

No errors found!

**** REPORT ****
Total valid lines of data: 1000
Total invalid lines of data: 0
Total students with high scores (>80): 346
Mean (average) score: 75.604
Max score: 100
Min score: 40
Range of scores: 60
Median score: 77.0
Question that most people skip: 6 - 117 - 0.117, 23 - 117 - 0.117
Question that most people answer incorrectly: 12 - 130 - 0.13
